In [45]:
import matplotlib.pyplot as plt
import numpy as np

import pint
import pint_pandas

import pandas as pd

pd.options.mode.copy_on_write = True


class LabCtx:
    _ureg = pint.UnitRegistry(autoconvert_offset_to_baseunit=True)
    _Q = _ureg.Quantity
    pint_pandas.PintType.ureg = _ureg

    _ureg.enable_contexts("Gaussian")

    def __init__(self, exelPath: str) -> None:
        self._labData = pd.read_excel(exelPath, sheet_name=None, header=[0, 1])

    def _getTable(self, name: str) -> pd.DataFrame:
        return self._labData[name].pint.quantify(level=-1)

    def _toLatexFormat(self):
        self._ureg.default_format = "Lx"

    def _toDefaultFormat(self):
        self._ureg.default_format = "P"


Q = LabCtx._Q

UR = LabCtx._ureg


def q_for_eval(magnitude, unit=None):
    return Q(magnitude, unit)


class LabTable:
    def __init__(
        self, ctx: LabCtx, name: str = "", table: pd.DataFrame = pd.DataFrame(data={})
    ) -> None:

        self.ctx = ctx

        if not table.empty:
            self._table = table
            return

        self._table = self.ctx._getTable(name)

    def toLatex(
        self,
        file: str,
        columnsRen: dict = {},
        caption="",
        float_format="%.2f",
        toStdout=False,
    ) -> None:
        self.ctx._toLatexFormat()

        ltbl = self._table[list(columnsRen.keys())]

        ltbl.rename(columns=columnsRen).pint.dequantify().to_latex(
            file,
            index=False,
            caption=caption,
            position="H",
            float_format=float_format,
        )

        self.ctx._toDefaultFormat()

        if toStdout:
            print(ltbl.rename(columns=columnsRen).pint.dequantify())

    def setColUnits(self, col: str, u: pint.Unit | str):
        self._table[col] = self._table[col].pint.to(u)
        return self

    def getColUnits(self, col: str) -> pint.Unit:
        return self._table[col].pint.units

    def addCol(self, col: str, data: pd.Series) -> None:
        self._table[col] = data

    def col(self, col: str) -> pd.Series:
        return self._table[col]

    def colQuantity(self, col: str) -> pint.Quantity:
        return self._table[col].values.quantity

    def colRaw(self, col: str) -> list:
        return self._table[col].values.quantity.magnitude

    def query(self, q: str):
        return LabTable(self.ctx, table=self._table.query(q))

    def transmuteArrayCol(self, col: str, ncol: str, func):
        res = []
        cols = self.colRaw(col)

        for i in range(len(cols)):
            arr = np.fromstring(cols[i], sep=" ")
            res.append(func(arr))

        self.addCol(ncol, pd.Series(res, dtype=f"pint[{str(self.getColUnits(col))}]"))

    def printPretty(self):
        print(self._table)


class LabPlot:
    def __init__(self, file: str, xlbl: str, ylbl: str) -> None:
        self.file = file

        self.xlbl = xlbl
        self.ylbl = ylbl

    def __enter__(self):
        self.fig, self.axis = plt.subplots(figsize=(12, 10), dpi=100)
        self.axis.grid(True)

        self.axis.set_xlabel(self.xlbl, fontsize=20)
        self.axis.set_ylabel(self.ylbl, fontsize=20)

        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.axis.legend()
        self.fig.savefig(self.file)


def plotPolyfitWeights(
    ax,
    x,
    y,
    sigma,
    name: str,
    scatter=True,
    prec=5,
):

    fit, pcov = np.polyfit(x, y, 1, w=(1 / sigma), full=False, cov=True)
    perr = np.sqrt(np.diag(pcov))
    poly = np.poly1d(fit)

    xsp = np.linspace(min(x), max(x), 50)
    ax.plot(
        xsp,
        poly(xsp),
        label=f"{name}: ${fit[0]:.{prec}f}\\pm{perr[0]:.{prec}f} + {fit[1]:.{prec}f}\\pm{perr[1]:.{prec}f}$",
    )

    if scatter:
        ax.errorbar(x, y, xerr=sigma, fmt="o")


def plotMNK(ax, x_units, y_units, name, scatter=False, prec=5) -> list:
    x = x_units
    y = y_units

    fit, pcov = np.polyfit(x, y, 1, full=False, cov=True)
    perr = np.sqrt(np.diag(pcov))
    poly = np.poly1d(fit)

    x = np.linspace(min(x), max(x), 50)
    ax.plot(
        x,
        poly(x),
        label=f"{name}: ${fit[0]:.{prec}f}\\pm{perr[0]:.{prec}f}$",
    )

    if scatter:
        ax.scatter(x_units, y_units)

    return fit

In [2]:
labCtx = LabCtx("lab8.1.xlsx")

In [3]:
table1 = LabTable(labCtx, name="1")

In [ ]:
table1.printPretty()

In [ ]:
table1.addCol("W", table1.col("I") * table1.col("U"))
table1.setColUnits("W", "watt")

In [6]:
table1.transmuteArrayCol("T", "Tmean", np.mean)

table1.transmuteArrayCol("T", "Tsigma", np.std)

In [7]:
table1.addCol("Tt", 1.05 * table1.col("Tmean"))

In [ ]:
table1.toLatex(
    "tex/tables/t1.tex",
    {
        "I": r"$I$",
        "U": r"$U$",
        "W": r"$W$",
        "Tmean": r"$T_\text{light}$",
        "Tsigma": r"$\sigma_t$",
        "Tt": r"$T$",
    },
    float_format="%.2f",
)

In [ ]:
with LabPlot("tex/pictures/1.png", "T, C", "W, Вт") as plot:

    plot.axis.errorbar(
        table1.colRaw("Tt"),
        table1.colRaw("W"),
        xerr=table1.colRaw("Tsigma"),
        fmt="o",
    )

In [ ]:
with LabPlot("tex/pictures/2.png", "lnT", "lnW") as plot:

    plotPolyfitWeights(
        plot.axis,
        np.log(table1.colRaw("Tt")),
        np.log(table1.colRaw("W")),
        np.log(table1.colRaw("Tsigma")) / 100,
        "T log",
    )

In [44]:
table1.setColUnits("Tt", "degK")


tbl1500 = table1.query(f"Tt >= @q_for_eval(1700, 'degK')")

# tbl1500.printPretty()

eT = [0.209, 0.223, 0.236, 0.249, 0.249, 0.249, 0.249]

S = Q(0.36, "cm**2")

tbl1500.addCol("Bolzman", tbl1500.col("W") / (eT * S * tbl1500.col("Tt") ** 4))

tbl1500.toLatex(
    "tex/tables/t2.tex",
    {
        "Tt": r"$T$",
        "Bolzman": r"$\sigma_B$",
    },
    float_format="%.2E",
)


print(np.mean(tbl1500.col("Bolzman")))
print(np.std(tbl1500.col("Bolzman")))

8.245548550396931×10⁻¹⁴ watt/centimeter²/kelvin⁴
1.4468502499606157×10⁻¹⁵ watt/centimeter²/kelvin⁴


/var/folders/fz/4mj7qkt17hxfqzw5cc4s9zg80000gn/T/ipykernel_3901/1120612410.py:26: DeprecationWarning: This function will be removed in future versions of pint.
Use ureg.formatter.default_format
  self._ureg.default_format = "Lx"
/var/folders/fz/4mj7qkt17hxfqzw5cc4s9zg80000gn/T/ipykernel_3901/1120612410.py:29: DeprecationWarning: This function will be removed in future versions of pint.
Use ureg.formatter.default_format
  self._ureg.default_format = "P"


In [69]:
Bolz = Q("8.27*10**-12  watt * cm**(-2) * degK**(-4)")

plank = np.cbrt(
    (2 * np.pi**5 * UR.boltzmann_constant**4) / (15 * UR.speed_of_light**2 * Bolz)
)

# pq = Q(plank.to_root_units().magnitude, "gram * meter**2 / s")
# pq.to("joule*s")
pb = plank.to_base_units()
pq = Q(pb.magnitude, str(pb.units))
pq.to("joule*s")

<Quantity(5.84285047e-34, 'joule * second')>

In [ ]:
a = -28
eT1 = 0.232

np.exp(a) / (eT1 * 0.36)

In [70]:
tp = [930, 942, 967, 1000, 985, 972]

print(np.mean(tp))
print(np.std(tp))

966.0
23.895606290697042


In [78]:
va = Q(38, "mvolt")
p = Q(41, "mcvolt/degC")


(va / p).to("delta_degree_Celsius")

<Quantity(926.829268, 'delta_degree_Celsius')>